In [10]:
import praw
import pandas as pd
import time
import re
import random

In [11]:
reddit = praw.Reddit(
    client_id = "yeErT4HRa1wl_1wHreQsbw",
    client_secret = "PLF0MvtZCg2X-M6LmjZ_ECnYghqOTA",
    user_agent= "UserAgent"
)

In [12]:
# --- Categories and related subreddits ---
categories = {
    "education": ["education", "college", "university"],
    "health": ["health", "medicine", "HealthInsurance"],
    "insurance": ["insurance", "CarInsurance"],
    "banking": ["banking", "CreditCards", "personalfinance"],
    "housing": ["Landlord", "RealEstate", "housing"],
    "transport": ["cars", "publictransport", "travel"],
    "shopping": ["shopping", "customerService", "amazon"],
    "technology": ["techsupport", "technology", "computers"],
    "government": ["legaladvice", "tax", "publicpolicy"]
}

# --- Problem keywords for each category ---
problem_keywords = {
    "education": {
        "tuition fees": ["tuition", "fees", "expensive", "payment"],
        "exam stress": ["exam", "test", "stress", "grade"],
        "poor teaching": ["teacher", "professor", "bad lecture", "unclear"],
        "course registration": ["registration", "enroll", "system error"],
        "online learning issues": ["online class", "zoom", "connection"]
    },
    "health": {
        "hospital delay": ["hospital", "waiting", "delay", "appointment"],
        "insurance claim": ["insurance", "claim", "rejected", "coverage"],
        "wrong diagnosis": ["diagnosis", "misdiagnosed", "doctor mistake"],
        "medicine shortage": ["medicine", "drug", "unavailable"],
        "emergency response": ["ambulance", "emergency", "slow response"]
    },
    "insurance": {
        "claim denied": ["claim denied", "rejected", "refused"],
        "slow response": ["delay", "waiting", "pending"],
        "policy confusion": ["policy", "terms", "coverage", "confusing"],
        "fraud issues": ["fraud", "scam", "fake policy"]
    },
    "banking": {
        "account locked": ["locked", "blocked", "account access"],
        "fraud transaction": ["unauthorized", "fraud", "scam"],
        "credit card issues": ["credit card", "declined", "limit"],
        "loan problems": ["loan", "application", "rejected"],
        "atm error": ["atm", "cash", "withdraw", "error"]
    },
    "housing": {
        "rent increase": ["rent", "increase", "raise", "expensive"],
        "maintenance issue": ["repair", "broken", "leak", "maintenance"],
        "landlord problem": ["landlord", "owner", "refuse", "deposit"],
        "eviction": ["evicted", "notice", "remove"],
        "neighbor issue": ["noise", "neighbor", "complaint"]
    },
    "transport": {
        "traffic jam": ["traffic", "jam", "congestion", "stuck"],
        "delayed bus": ["bus", "late", "delay"],
        "road accident": ["accident", "crash", "collision"],
        "fuel price": ["fuel", "gas", "price"],
        "flight cancellation": ["flight", "cancelled", "airport"]
    },
    "shopping": {
        "late delivery": ["late", "delay", "shipping"],
        "refund issues": ["refund", "money back", "return"],
        "fake product": ["fake", "counterfeit", "scam"],
        "poor quality": ["quality", "broken", "damaged"],
        "bad customer service": ["support", "rude", "service"]
    },
    "technology": {
        "software bug": ["bug", "crash", "error", "glitch"],
        "hardware failure": ["hardware", "laptop", "broken", "keyboard"],
        "slow performance": ["slow", "lag", "freeze"],
        "network problem": ["wifi", "connection", "internet", "signal"],
        "data loss": ["data", "lost", "deleted"]
    },
    "government": {
        "tax issue": ["tax", "refund", "irs"],
        "public service delay": ["delay", "service", "application"],
        "corruption": ["bribe", "corruption", "fraud"],
        "legal confusion": ["law", "rule", "confusing"],
        "document processing": ["passport", "id", "document"]
    }
}

# --- Parameters ---
limit_per_subreddit = 100000
data = []

# --- Text cleaner ---
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^A-Za-z0-9\s.,!?']", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# --- Problem detector ---
def detect_problem(category, text):
    text_lower = text.lower()
    for problem, keywords in problem_keywords.get(category, {}).items():
        if any(kw in text_lower for kw in keywords):
            return problem
    return "other"

# --- Collect posts ---
for category, subs in categories.items():
    print(f"\n📂 Collecting category: {category}")
    for sub in subs:
        print(f"   🔸 Subreddit: r/{sub}")
        try:
            subreddit = reddit.subreddit(sub)
            for post in subreddit.top(limit=limit_per_subreddit):
                if post.selftext and len(post.selftext) > 50:
                    problem_type = detect_problem(category, post.selftext)
                    data.append({
                        "category": category,
                        "subreddit": sub,
                        "problem_type": problem_type,
                        "title": clean_text(post.title),
                        "text": clean_text(post.selftext)
                    })
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ Error in r/{sub}: {e}")

# --- Save to CSV ---
df = pd.DataFrame(data)
df.to_csv("reddit_complaints_dataset.csv", index=False, encoding="utf-8")
print(f"\n✅ Saved {len(df)} posts with categories and problem types to reddit_complaints_dataset.csv")

df.head()



📂 Collecting category: education
   🔸 Subreddit: r/education
   🔸 Subreddit: r/college
   🔸 Subreddit: r/university

📂 Collecting category: health
   🔸 Subreddit: r/health
   🔸 Subreddit: r/medicine
   🔸 Subreddit: r/HealthInsurance

📂 Collecting category: insurance
   🔸 Subreddit: r/insurance
   🔸 Subreddit: r/CarInsurance
⚠️ Error in r/CarInsurance: received 404 HTTP response

📂 Collecting category: banking
   🔸 Subreddit: r/banking
   🔸 Subreddit: r/CreditCards
   🔸 Subreddit: r/personalfinance

📂 Collecting category: housing
   🔸 Subreddit: r/Landlord
   🔸 Subreddit: r/RealEstate
   🔸 Subreddit: r/housing

📂 Collecting category: transport
   🔸 Subreddit: r/cars
   🔸 Subreddit: r/publictransport
⚠️ Error in r/publictransport: received 404 HTTP response
   🔸 Subreddit: r/travel

📂 Collecting category: shopping
   🔸 Subreddit: r/shopping
⚠️ Error in r/shopping: received 403 HTTP response
   🔸 Subreddit: r/customerService
   🔸 Subreddit: r/amazon

📂 Collecting category: technology
   

,category,subreddit,problem_type,title,text
0,education,education,other,This is the Bill that is Shutting down the DOE...,Someone please read this. It made me dry heave...
1,education,education,poor teaching,Will Tim Walz bring a renewed interest in educ...,"Tim Walz former teacher, football coach, and v..."
2,education,education,other,Trump and Elon have kids in private school of ...,But for everyone who doesn t have the money to...
3,education,education,exam stress,"The school says my 14 year old is doing great,...","I live in the U.S., in a community with better..."
4,education,education,other,"Holy crap, watching kids copy paste essays in ...",So draftback went paid and I was pissed becaus...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3856 entries, 0 to 3855
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      3856 non-null   object
 1   subreddit     3856 non-null   object
 2   problem_type  3856 non-null   object
 3   title         3856 non-null   object
 4   text          3856 non-null   object
dtypes: object(5)
memory usage: 150.8+ KB
